In [1]:
import pandas as pd
import numpy as np

In [2]:
features_target_train = pd.read_csv("data/features_target_train.csv")
features_target_test = pd.read_csv("data/features_target_test.csv")

In [3]:
columns = features_target_train.columns

In [4]:
X_train = features_target_train[columns[:-1]]
X_test = features_target_test[columns[:-1]]

y_train = features_target_train[columns[-1]]
y_test = features_target_test[columns[-1]]

# Check for zero variance columns

In [5]:
for column, variance in X_train.var().items():
    if variance == 0:
        print(column, variance)

SMR_VSA8 0.0
SlogP_VSA9 0.0
fr_diazo 0.0
fr_prisulfonamd 0.0
